In [25]:
# ─── ZELLE 0: Abhängigkeiten installieren ───
import sys

# installiere alles Notwendige direkt in der laufenden Kernel-Umgebung
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install numpy torch torchvision matplotlib

import numpy as np
print("✅ NumPy:", np.__version__)


✅ NumPy: 2.2.4


In [26]:
import sys
print("Notebook läuft unter Python:", sys.executable)


Notebook läuft unter Python: /home/codespace/.python/current/bin/python


In [27]:
# ─── ZELLE 0: NumPy installieren und importieren ───
# (notwendig, damit torchvision intern auf np.array zugreifen kann)
try:
    import numpy as np
except ModuleNotFoundError:
    # installiert es direkt im laufenden Notebook-Interpreter
    import sys
    !{sys.executable} -m pip install numpy
    import numpy as np

print("NumPy Version:", np.__version__)


NumPy Version: 2.2.4


In [28]:
import os
print("📂 Aktueller Notebook-Pfad:", os.getcwd())


📂 Aktueller Notebook-Pfad: /workspaces/AI_Project_Fruits/fruit_classification/notebook


# 🍎 Fruchtklassifikation mit Transfer Learning (PyTorch + ResNet18)

## Projektbeschreibung
In diesem Projekt trainieren wir ein Bildklassifikationsmodell, das fünf Obstsorten erkennt:
- Apfel
- Banane
- Erdbeere
- Kirsche
- Orange

Wir nutzen Transfer Learning mit einem vortrainierten ResNet18-Modell von torchvision.


In [29]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# Gerät wählen
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Verwende Gerät:", device)


Verwende Gerät: cpu


## 📂 Daten laden & vorbereiten
Wir definieren die Datenpfade und setzen die Transformationspipeline zur Bildvorverarbeitung.


In [30]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

print("📂 Aktuelles Notebook-Verzeichnis:", os.getcwd())
data_dir = "../dataset"   # ← hier angepasst

train_path = os.path.join(data_dir, "train")
test_path  = os.path.join(data_dir, "test")

print("Train-Pfad existiert?", os.path.isdir(train_path))
print("Test-Pfad existiert?",  os.path.isdir(test_path))
print("Inhalt train/:", os.listdir(train_path) if os.path.isdir(train_path) else "– nicht gefunden")
print("Inhalt test/:",  os.listdir(test_path)  if os.path.isdir(test_path) else "– nicht gefunden")

# wenn beides True ist, fahre fort:
train_data = datasets.ImageFolder(train_path, transform=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
]))
val_data = datasets.ImageFolder(test_path, transform=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
]))

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_data,   batch_size=32)

print("✅ Klassen:", train_data.classes)


📂 Aktuelles Notebook-Verzeichnis: /workspaces/AI_Project_Fruits/fruit_classification/notebook
Train-Pfad existiert? True
Test-Pfad existiert? True
Inhalt train/: ['Banane', 'Orange', 'Kirsche', 'Apfel', 'Erdbeere']
Inhalt test/: ['Banane', 'Orange', 'Kirsche', 'Apfel', 'Erdbeere']
✅ Klassen: ['Apfel', 'Banane', 'Erdbeere', 'Kirsche', 'Orange']


In [31]:
# ————— ZELLE: Klassen ermitteln —————
class_names = train_data.classes
print("Gefundene Klassen:", class_names)


Gefundene Klassen: ['Apfel', 'Banane', 'Erdbeere', 'Kirsche', 'Orange']


## 🧠 Klassifizierungsmodell definieren
Wir definieren ein Transfer-Learning-Modell auf Basis von ResNet18.


In [32]:
class FruitClassifier(nn.Module):
    def __init__(self, num_classes):
        super(FruitClassifier, self).__init__()
        self.model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

model = FruitClassifier(num_classes=len(class_names)).to(device)


## ⚙️ Trainingskonfiguration
Wir definieren die Loss-Funktion, den Optimierer und den Lernraten-Scheduler.


In [33]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


## 🔁 Trainings- & Validierungsschleife
Wir erstellen eine Funktion, um das Modell über mehrere Epochen zu trainieren und zu validieren.


In [34]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f"Epoche {epoch+1}/{num_epochs}")
        print("-" * 10)

        for phase in ['train', 'val']:
            model.train() if phase == 'train' else model.eval()
            dataloader = train_loader if phase == 'train' else val_loader

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_corrects.double() / len(dataloader.dataset)

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

    print(f"Bestes Modell mit Genauigkeit: {best_acc:.4f}")
    model.load_state_dict(best_model_wts)
    return model


## 🏋️ Modell trainieren
Wir starten das Training des Modells über 10 Epochen.


In [39]:
model = train_model(model, criterion, optimizer, scheduler, num_epochs=10)


Epoche 1/10
----------


RuntimeError: Numpy is not available

## 💾 Modell speichern
Wir speichern das trainierte Modell als `.pt`-Datei.


In [ ]:
model_path = "../fruit_classification/fruit_model.pt"
torch.save(model.state_dict(), model_path)
print("✅ Modell gespeichert unter:", model_path)


## 📊 Modell auf Testdaten evaluieren
Wir berechnen die Gesamtgenauigkeit des Modells auf den Testdaten.


In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Genauigkeit auf Testdaten: {100 * correct / total:.2f}%")
